In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import json
import timeit

In [12]:
#LOADING THE CSV FILE DATA
df = pd.read_csv("amazon.csv")
print(df)

urls = [] #list to store the urls

for index, rows in df.iterrows():
    URL = f"https://www.amazon.{rows.country}/dp/{rows.Asin}"
    urls.append(URL)
        
    
#SETTING UP BS4
headers =  {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.81 Safari/537.36"}


#FUNCTIONS TO EXTRACT DATA

def get_description(soup):
    description = soup.find(id=["featurebullets_feature_div", "bookDescription_feature_div"]).get_text().strip()
    return description

def get_image_link(soup):
    img_link = soup.find(id = ["imgTagWrapperId","img-canvas"]).img["src"]
    return img_link
    
def get_price(soup):
    try:
        price = soup.find("span", attrs = {"class" : ["a-price", "a-color-price"]}).get_text().strip() 
    except AttributeError:
        price = "NA"
    return price
        
def get_title(soup):
    if page.status_code == 200: #not 404 error
        title = soup.find(id="productTitle").get_text().strip()
        return title

    
        

     Unnamed: 0       id        Asin country
0             0        1        1015      de
1             1  2424796        1015      fr
2             2        2  000004458X      de
3             3  2424797  000004458X      fr
4             4        3     1002198      de
..          ...      ...         ...     ...
995         995  2425286     4484924      fr
996         996      449     4485742      de
997         997  2425287     4485742      fr
998         998      450     4486072      de
999         999  2425288     4486072      fr

[1000 rows x 4 columns]


In [15]:
dictionary = {}

i = 0
for url in urls:
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, "lxml")
    if(get_title(soup)==None):
        pass
    else:
        i += 1
        dictionary[i] = [url, get_title(soup), get_price(soup), get_image_link(soup), get_description(soup)]

with open('output.json', 'w') as json_file:
  json.dump(dictionary, json_file, indent = 4)
       
        

In [37]:
#Reading the output json file to insert values to sqlite database

with open("output.json", "r") as f:
    data = json.load(f)

data_list = []
for key, values in data.items():
    data_list.append(tuple(values))


Old Spice Rasur Creme - 70 G (original) - Packung Von 2


In [39]:
#Creating ans inserting values to the database from the json file

import sqlite3 as sql

conn = sql.connect("output.db")
cur = conn.cursor()

cur.execute("""CREATE TABLE IF NOT EXISTS newoutput
                    (url text PRIMARY KEY, title text, price text, img_src text, description text)""")

cur.executemany("INSERT OR IGNORE INTO newoutput VALUES (?,?,?,?,?)", data_list)

conn.commit()


#Prints the database values

# for row in cur.execute("""SELECT * FROM newoutput"""):
#     print(row)
